## Upgrade the script to allow re-start

#### Communicate with the Imperial-HPC staff, informed that it is risky to run simulation in one go longer than 72 hours, even if we can pay for it, as the HPC system will be maintained from time to time, and may break down.
#### Instead, should try to save and restart, which requires some upgrades to the script. 

### Problems to think: 
#### What is needed to be saved for restart?
#### How many ageing sets we should aim?
#### Shall we carry all the solution object all the way down? (doesn’t sound like a good idea, should only carry my self-defined dictionary object, and the last solution)
#### Make sure we have a global variable to check how many ageing sets have been run.
#### How to get informed when the simulation is finished?

In [71]:
# quick thing to try: reload one just finished and check what have been saved, 
# write a new function in Fun_P2.py call Run_restart

In [72]:
# magic lines that avoid re-start 
%load_ext autoreload
%autoreload 2
import pybamm as pb;import pandas as pd;import numpy as np;
import os, json,openpyxl,traceback,multiprocessing,scipy.optimize,sys
import matplotlib.pyplot as plt;
import pickle,imageio,timeit,random,time, signal
from scipy.io import savemat,loadmat;
from pybamm import constants,exp;import matplotlib as mpl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
########################     Global settings!!!
rows_per_file = 1;  Scan_end_end = 75;
purpose_i = "Full_5Exp_5Ts"  
Re_No = 1    # update 240110 for restart

On_HPC =  False  
Runshort  =   False                    # a long run or a quick test
Add_Rest = False
i_bundle = 54

Scan_start = (i_bundle-1)*rows_per_file+1;    
Scan_end   = min(Scan_start + rows_per_file-1, Scan_end_end)    
purpose = f"{purpose_i}_Case_{Scan_start}_{Scan_end}"
# interpetation: Simnon suggested, with cracking activation, heat transfer
para_csv = f"Bundle_{i_bundle}.csv"  # name of the random file to get parameters


if On_HPC:                          # Run on HPC
    Path_csv = f"InputData/{purpose_i}/" 
    Path_NiallDMA = "InputData/" 
    BasicPath=os.getcwd() 
    Para_file = Path_csv +  para_csv
else:
    import sys  
    str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
    str_path_1 = os.path.abspath(
        os.path.join(str_path_0,"wip/Rio_Code/Fun_P2"))
    sys.path.append(str_path_1) 
    Path_NiallDMA = os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/InputData/") # for Linux
    BasicPath =  os.path.expanduser(
        "~/EnvPBGEM_Linux/SimSave/P2_R9_Dim")
    Para_file = BasicPath+f'/Get_Random_sets2/{purpose_i}/'+para_csv
    
if not os.path.exists(BasicPath +"/"+ purpose):
    os.mkdir(BasicPath +"/"+ purpose);
# import all functions 
from Fun_P2 import * 

# Load the random file
parameter_names, combinations = load_combinations_from_csv(Para_file)


pool_no = len(combinations)
Indexs  = np.arange(Scan_start-1,Scan_end)
index_list = Indexs+1
""" para_dict_Same = {
   "Total ageing cycles":int(tot_cyc), # 
   "Ageing cycles between RPT":int(cyc_age), # 
   "Update cycles for ageing": int(cyc_age),
} """
# Get all para
Para_dict_list = []
# get all dictionaries
for combination in combinations:
    input_dict = {}
    for parameter_name,para_value in zip(parameter_names,combination ):
        input_dict[parameter_name] = para_value
    Para_dict_list.append(input_dict)
print(f"Total scan case is {len(Para_dict_list)}")

Total scan case is 1


In [74]:
Para_dict_list[0]

{'Scan No': 54,
 'Exp No.': 8,
 'Ageing temperature': 25.0,
 'Mesh list': '[5,5,5,60,20]',
 'Inner SEI lithium interstitial diffusivity [m2.s-1]': 9.81e-19,
 'Dead lithium decay constant [s-1]': 1e-07,
 'Lithium plating kinetic rate constant [m.s-1]': 1e-14,
 'Negative electrode LAM constant proportional term [s-1]': 2.84e-09,
 'Positive electrode LAM constant proportional term [s-1]': 2.98e-18,
 'Negative electrode cracking rate': 5.29e-25,
 'Outer SEI partial molar volume [m3.mol-1]': 5.22e-05,
 'SEI growth activation energy [J.mol-1]': 500.0,
 'Negative cracking growth activation energy [J.mol-1]': 0.0,
 'Negative electrode diffusivity activation energy [J.mol-1]': 60000.0,
 'Positive electrode diffusivity activation energy [J.mol-1]': 12000.0,
 'Contact resistance [Ohm]': 0.0115,
 'Total heat transfer coefficient [W.m-2.K-1]': 20,
 'Initial electrolyte excessive amount ratio': 10,
 'Cycles within RPT': 1,
 'RPT temperature': 25,
 'Para_Set': 'OKane2023',
 'Model option': "{'SEI': '

In [75]:
# define global index and dict for all experiment data - prepare for read!
Exp_Path = [
    "Expt 1 - Si-based Degradation/",
    "Expt 2,2 - C-based Degradation 2/",
    "Expt 3 - Cathode Degradation and Li-Plating/",
    "Expt 4 - Drive Cycle Aging (Control)/",
    "Expt 5 - Standard Cycle Aging (Control)/",]
Exp_head = [
    "Expt 1",
    "Expt 2,2",
    "Expt 3",
    "Expt 4",
    "Expt 5",]
Exp_1_Cell = ["A","B","J","D","E","F","K","L","M"];
Exp_1_Temp = {
    "A":"10","B":"10","J":"10",
    "D":"25","E":"25","F":"25",
    "K":"40","L":"40","M":"40",}
Temp_Cell_Exp_1 = {
    "10":["A","B","J"],
    "25":["D","E","F"],
    "40":["K","L","M"],}
Exp_2_Cell = ["A","B","C","D","E","F"];
Exp_2_Temp = {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",}
Temp_Cell_Exp_2 = {
    "10":["A","B"],
    "25":["C","D"],
    "40":["E","F"],}
Exp_3_Cell = ["A","B","C","D","E","F","G","H","I"];
Exp_3_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25","F":"25",
    "G":"40","H":"40","I":"40"}
Temp_Cell_Exp_3 = {
    "10":["A","B","C"],
    "25":["D","E","F"],
    "40":["G","H","I"],}
Exp_4_Cell = ["A","B","C","D","E","F","G","H"];
Exp_4_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_4 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_5_Cell = ["A","B","C","D","E","F","G","H"];
Exp_5_Temp = {
    "A":"10","B":"10","C":"10",
    "D":"25","E":"25",
    "F":"40","G":"40","H":"40",}
Temp_Cell_Exp_5 = {
    "10":["A","B","C"],
    "25":["D","E",],
    "40":["F","G","H"],}
Exp_All_Cell  = [Exp_1_Cell,Exp_2_Cell,Exp_3_Cell,Exp_4_Cell,Exp_5_Cell]
Exp_Temp_Cell = [Exp_1_Temp,Exp_2_Temp,Exp_3_Temp,Exp_4_Temp,Exp_5_Temp]
Temp_Cell_Exp_All = [Temp_Cell_Exp_1,Temp_Cell_Exp_2,Temp_Cell_Exp_3,Temp_Cell_Exp_4,Temp_Cell_Exp_5]
Mark_Cell_All = [
    {
    "A":"o","B":">","J":"v",
    "D":"o","E":">","F":"v",
    "K":"o","L":">","M":"v",},
    {
    "A":"10","B":"10",
    "C":"25","D":"25",
    "E":"40","F":"40",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">","F":"v",
    "G":"o","H":">","I":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",},
    {
    "A":"o","B":">","C":"v",
    "D":"o","E":">",
    "F":"o","G":">","H":"v",}]
Color_Cell_All = [
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"J":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "K":[1,0,0,0.4],"L":[1,0,0,0.4],"M":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"C":[0, 0, 0,0.7],
    "E":[1,0,0,0.4],"F":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],"F":[0, 0, 0,0.7],
    "G":[1,0,0,0.4],"H":[1,0,0,0.4],"I":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],},
    {
    "A":[2/255, 3/255, 226/255,0.7],"B":[2/255, 3/255, 226/255,0.7],"C":[2/255, 3/255, 226/255,0.7],
    "D":[0, 0, 0,0.7],"E":[0, 0, 0,0.7],
    "F":[1,0,0,0.4],"G":[1,0,0,0.4],"H":[1,0,0,0.4],}]
# Update 23-06-17 
#   para="Total heat transfer coefficient [W.m-2.K-1]";
#   output="Volume-averaged cell temperature [C]"
########################  Output  ########################
keys_loc_RPT = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    #"CCend Negative electrode roughness ratio",
    #"CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    #"CDend Negative electrode roughness ratio",
    #"CDend Total SEI on cracks thickness [m]",
    #"REend Total SEI on cracks thickness [m]",
]
keys_tim_RPT = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "CD Anode potential [V]",    # self defined
    "CD Cathode potential [V]",  # self defined
    "CC Time [h]",
    "CC Terminal voltage [V]",
    "CC Anode potential [V]",    # self defined
    "CC Cathode potential [V]",  # self defined
]
keys_cyc_RPT = [   # default: CDend
    "Discharge capacity [A.h]",
    "Throughput capacity [A.h]",
    "CDend Total lithium capacity in particles [A.h]",
    "CDend Loss of capacity to lithium plating [A.h]",
    "CDend Loss of capacity to SEI [A.h]",
    "CDend Loss of capacity to SEI on cracks [A.h]",
    #"CDend X-averaged total SEI on cracks thickness [m]",
    #"CDend X-averaged negative electrode roughness ratio",
    "CDend Local ECM resistance [Ohm]",
    "CDsta Negative electrode stoichiometry", 
    "CDend Negative electrode stoichiometry",
    "CDsta Positive electrode stoichiometry", 
    "CDend Positive electrode stoichiometry",
    "CDend Negative electrode capacity [A.h]",
    "CDend Positive electrode capacity [A.h]",
]

keys_loc_AGE = [ # MAY WANT TO SELECT AGEING CYCLE later
    # Default output:
    "x [m]",
    "x_n [m]",
    "x_s [m]",
    "x_p [m]",
    # default: end; 
    "CCend Porosity",
    "CCend Negative electrode interfacial current density [A.m-2]",
    "CCend Electrolyte potential [V]",
    "CCend Electrolyte concentration [mol.m-3]",
    "CCend Negative electrode reaction overpotential [V]",
    "CCend Negative particle surface concentration [mol.m-3]",
    "CCend Negative electrode surface potential difference [V]",
    "CCend SEI film overpotential [V]",
    #"CCend Negative electrode roughness ratio",
    #"CCend Total SEI on cracks thickness [m]",

    "CDend Porosity",
    "CDend Negative electrode interfacial current density [A.m-2]",
    "CDend Electrolyte potential [V]",
    "CDend Electrolyte concentration [mol.m-3]",
    "CDend Negative electrode reaction overpotential [V]",
    "CDend Negative particle surface concentration [mol.m-3]",
    #"CDend Negative electrode roughness ratio",
    #"CDend Total SEI on cracks thickness [m]",
    "CDend Negative electrode surface potential difference [V]",
    "CDend SEI film overpotential [V]",
    "CDend Electrolyte diffusivity [m2.s-1]",
    "CDend Electrolyte conductivity [S.m-1]",
]
keys_tim_AGE = [
    # default: CD
    "CD Time [h]",
    "CD Terminal voltage [V]",
    "CD Anode potential [V]",    # self defined
    "CD Cathode potential [V]",  # self defined
    
    "CC Time [h]",
    "CC Terminal voltage [V]",
    "CC Anode potential [V]",    # self defined
    "CC Cathode potential [V]",  # self defined
]
""" # update 240110 - need to add " (Age)" in the end!
for i in range(len(keys_tim_AGE)):
    keys_tim_AGE[i] = keys_tim_AGE[i] + " (Age)"
# 
for i in range(len(keys_loc_AGE)):
    keys_loc_AGE[i] = keys_loc_AGE[i] + " (Age)" """
    
keys_cyc_AGE = [];
keys_all_RPT = [keys_loc_RPT,keys_tim_RPT,keys_cyc_RPT];
keys_all_AGE = [keys_loc_AGE,keys_tim_AGE,keys_cyc_AGE];
keys_all = [keys_all_RPT,keys_all_AGE];


# Write para - 1st round:
Values_1 = []
head_keys = list(Para_dict_list[0].keys())
head_pre = [
    "Scan No","Exp No.","Y or N",
    "Error Tot%","Error SOH %","Error LLI %",
    "Error LAM NE %","Error LAM PE %",
    "Error Res %","Error ageT %","Punish",
    "Dry out",]

head_pos = [ 
    "Total ageing cycles","Ageing cycles between RPT",
    "Update cycles for ageing",
    "exp_AGE_text", "exp_RPT_text",
   "Cap Loss","LLI to LiP",
   "LLI to SEI","LLI to sei-on-cracks",
   "LAM to Neg","LAM to Pos",
   "Vol_Elely_Tot Final", "Vol_Elely_JR Final","Width Final","Error"]
Values_1 .append([*head_pre,*head_keys,*head_pos])
book_name_xlsx = f'Summary_{purpose}_Re_{Re_No}.xlsx' # update 240110 for restart
sheet_name_xlsx = 'Output'
Target  = f'/{purpose}/'
write_excel_xlsx(
    BasicPath + Target + book_name_xlsx, 
    sheet_name_xlsx, Values_1)   
Exp_pack = [
    Exp_All_Cell,Temp_Cell_Exp_All,
    Exp_Path,Exp_head,Exp_Temp_Cell,
    book_name_xlsx,]  
# remaining set-up for experiment and create folders

if not os.path.exists(BasicPath +Target+"Mats"):
    os.mkdir(BasicPath +Target +"Mats");
if not os.path.exists(BasicPath +Target+"Plots"):
    os.mkdir(BasicPath +Target+"Plots");
if not os.path.exists(BasicPath +Target+"Excel"):
    os.mkdir(BasicPath +Target+"Excel");

Successfully create a excel file


In [76]:
# reload previous Re:

# [ midc_merge, dict_short, Paraupdate, Data_Pack, getSth] = Save_for_Reload

In [77]:
fs = 13; dpi = 100;
midc_merge_all = [];Sol_RPT_all = [];Sol_AGE_all = [];
Plot_Exp=True;     Timeout=True;     Timelimit = int(3600*48) # give 48 hours!
Return_Sol=True;   Check_Small_Time=True; R_from_GITT = False

# now assume we have: dict_short and midc_merge and Para_0_Dry_old
if Re_No == 0:
    midc_merge,Sol_RPT,Sol_AGE,DeBug_Lists = Run_P2_Excel (
        Para_dict_list[0],BasicPath, Path_NiallDMA, 
        purpose, Re_No,   Exp_pack, keys_all,dpi,fs,
        Runshort,   Plot_Exp,Timeout, Timelimit,
        Return_Sol, Check_Small_Time, R_from_GITT,Add_Rest,
    ) 
elif Re_No > 0:
    Scan_i = int(Para_dict_list[0][ 'Scan No'] )
    with open(
        BasicPath + Target+"Mats/" + str(Scan_i)
        + f'_Re_{Re_No-1}-Save_for_Reload.pkl', 'rb') as file:
        Save_for_Reload = pickle.load(file)
    midc_merge,Sol_RPT,Sol_AGE,DeBug_Lists = Run_Restart (
        Para_dict_list[0],BasicPath, Path_NiallDMA, Save_for_Reload,
        purpose, Re_No,   Exp_pack, keys_all,dpi,fs,
        Runshort,   Plot_Exp,Timeout, Timelimit,
        Return_Sol, Check_Small_Time, R_from_GITT,Add_Rest,
    ) 
else:
    print(f"Error! Re_No is {Re_No}")

Start Now! Scan 54 Re 1
Has Overwritten Initial outer SEI thickness [m] to be 6.73e-09 and Negative electrode porosity to be 0.235 to account for initial capacity loss of 0.135 Ah
Scan 54 Re 1: DryOut = On
Scan 54 Re 1: Spent 7.483 ms on Initialization
First run of Restart, Data_Pack and Paraupdate are already defined


At t = 513.346 and h = 5.50129e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 453.533 and h = 7.08224e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 213.532 and h = 6.24101e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 93.5324 and h = 6.07168e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 453.503 and h = 1.17618e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 213.503 and h = 5.81074e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 93.5033 and h = 5.11438e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 453.474 and h = 1.30043e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 213.474 and h = 5.42499e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 93.4743 and h = 3.09817e-12, the corrector convergence failed repeatedly or with |h| = hmin.


Must be the first run after restart, already have getSth
Succeed to run the ageing set for 78 cycles the 1th time
Scan 54 Re 1: Finish for No.2808 ageing cycles within 48 minutes, 53 seconds
Scan 54 Re 1: Finish post-process for No.2886 ageing cycles within 187.258 ms


At t = 229.266 and h = 1.80449e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 229.266 and h = 2.76837e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 220.555 and h = 2.63947e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 220.555 and h = 2.77197e-11, the corrector convergence failed repeatedly or with |h| = hmin.


Succeed to run RPT for the 1th time
Scan 54 Re 1: Finish for No.2886 RPT cycles within 2 minutes, 5 seconds
Scan 54 Re 1: Finish post-process for No.2886 RPT cycles within 814.429 ms


...

At t = 446.512 and h = 3.69223e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 206.512 and h = 4.98129e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 86.5118 and h = 1.1187e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 386.843 and h = 2.65109e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 146.843 and h = 3.54902e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 386.821 and h = 6.11567e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 146.821 and h = 3.47434e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 386.798 and h = 1.4072e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 146.798 and h = 5.91403e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 386.775 and h = 5.80968e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At

Succeed to run the ageing set for 78 cycles the 1th time
Scan 54 Re 1: Finish for No.5226 ageing cycles within 42 minutes, 25 seconds
Scan 54 Re 1: Finish post-process for No.5304 ageing cycles within 152.099 ms


At t = 304.266 and h = 1.72681e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 295.217 and h = 1.45804e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 295.217 and h = 2.08399e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 145.215 and h = 7.49462e-15, the corrector convergence failed repeatedly or with |h| = hmin.


Succeed to run RPT for the 1th time
Scan 54 Re 1: Finish for No.5304 RPT cycles within 2 minutes, 6 seconds
Scan 54 Re 1: Finish post-process for No.5304 RPT cycles within 655.370 ms


At t = 444.604 and h = 4.28518e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 204.604 and h = 6.95348e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 84.6038 and h = 5.51483e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 384.935 and h = 3.6898e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.935 and h = 4.81711e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 384.912 and h = 6.46126e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.913 and h = 4.20027e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 384.89 and h = 4.90733e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.89 and h = 4.81445e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 384.868 and h = 4.76277e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At 

In [ ]:
[my_dict_RPT, my_dict_AGE,mdic_dry] = midc_merge
my_dict_RPT['CDend SOH [%]']

[100.0,
 99.35482291189292,
 98.98258997043828,
 98.64080454996827,
 98.32255881412385,
 98.02323909420032,
 97.73960609785811,
 97.46924496044065,
 97.21040484322651,
 96.96168795158428,
 96.7219359884046,
 96.49012585527045,
 96.26540464132415,
 96.04707150271584,
 95.8345504424956,
 95.62732661697831,
 95.42501048441362,
 95.22720730140219,
 95.03362241110706,
 94.8439189251475,
 94.65789248666026,
 94.47531161987493,
 94.29594894992162,
 94.11964207596145,
 93.94621964767187,
 93.775111596372,
 93.60651096384909,
 93.44039474649045,
 93.27660553193039,
 93.11505683995064,
 92.95564360647379,
 92.79827695514986,
 92.64285750110002,
 92.48933708559164,
 92.337606869446,
 92.18764445801459,
 92.03932619574833]

In [ ]:
mdic_dry["Test_V_All"]

[0.0,
 -2.0952452127490308e-06,
 -3.36510400431882e-05,
 3.184166510346152e-05,
 -3.3568284898431765e-05,
 3.2338795735681226e-05,
 -3.35571521072831e-05,
 3.247892031731333e-05,
 -3.354328993124045e-05,
 3.258205081752611e-05,
 -3.352973975654469e-05,
 3.266042636884505e-05,
 -3.3513588801016064e-05,
 3.272225088242259e-05,
 -3.349309704536649e-05,
 3.276620776789598e-05,
 -3.347175076202693e-05,
 3.280186201866857e-05,
 -3.3450141191494036e-05,
 3.2827619110397574e-05,
 -3.344581998286631e-05,
 3.286486819852296e-05,
 -3.3428745663022124e-05,
 3.288705136268066e-05,
 -3.3413088299242735e-05,
 3.290391216806893e-05,
 -3.3396970041177724e-05,
 3.291904804642235e-05,
 -3.338082452238632e-05,
 3.293122100693292e-05,
 -3.336527479315646e-05,
 3.294169880352038e-05,
 -3.3354697820605675e-05,
 3.2954693398009444e-05,
 -3.334099647976801e-05,
 3.296210270561959e-05,
 -3.332487199707128e-05,
 3.296628371305197e-05,
 -3.3307800577806215e-05,
 3.296967649246992e-05,
 -3.329146105402196e-05,
 3.